In [ ]:
#Step 1: Import Necessary Libraries

import torch
from PIL import Image
from torchvision import transforms
import os
import sys
from IPython.display import display  # For displaying images in the notebook

# Adjust the path for custom modules
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import your model classes
# You may need to adjust sys.path or use specific import methods depending on your directory structure
from models.alexnet import AlexNet
from models.vgg import VGG
from models.resnet import ResNet
from models.efficientnet import EfficientNet
from models.vit import ViT


In [ ]:
# Step 2: Define Helper Functions

def select_model(model_name, num_classes):
    if model_name == 'AlexNet':
        return AlexNet(num_classes=num_classes)
    elif model_name == 'VGG':
        return VGG(num_classes=num_classes)
    elif model_name == 'ResNet':
        return ResNet(num_classes=num_classes)
    elif model_name == 'EfficientNet':
        return EfficientNet(num_classes=num_classes)
    elif model_name == 'ViT':
        return ViT(num_classes=num_classes)
    else:
        raise ValueError("Unknown model name")
    
def predict_image(image_path, model, device):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return predicted.item()    

In [ ]:
# Step 3: Set Up Device and Model Details

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 55  # Number of classes in your dataset
models_to_predict = ['AlexNet', 'VGG', 'ResNet', 'EfficientNet', 'ViT']


In [ ]:
#Step 4: Load Models and Make Predictions

test_dir = '../data/test'  # Adjust this path to your test images directory

for model_name in models_to_predict:
    print(f"Predicting with {model_name}...")
    model = select_model(model_name, num_classes)
    model_path = f"{model_name}.pth"  # Adjust path if necessary
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)

    for image_file in os.listdir(test_dir):
        if image_file.endswith('.png'):  # or other file types
            image_path = os.path.join(test_dir, image_file)
            prediction = predict_image(image_path, model, device)
            print(f"Model: {model_name}, Image: {image_file}, Predicted Class: {prediction}")
            display(Image.open(image_path))  # Display the image


In [ ]:
# Step5 Load the metrics of each model over epochs.

import matplotlib.pyplot as plt
import pandas as pd
import os

# Assuming the CSV files are saved in the 'output' directory
output_dir = '../output'  # Adjust this path if necessary

# List of models you've trained
models_to_train = ['AlexNet', 'VGG', 'ResNet', 'EfficientNet', 'ViT']

# Iterate over each model and plot its metrics
for model_name in models_to_train:
    metrics_file = os.path.join(output_dir, f"{model_name}_metrics.csv")
    
    # Check if the metrics file exists
    if not os.path.exists(metrics_file):
        print(f"No metrics file found for {model_name}.")
        continue
    
    # Read the metrics into a DataFrame
    df = pd.read_csv(metrics_file)
    
    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(df['Epoch'], df['Accuracy'], label='Accuracy')
    plt.plot(df['Epoch'], df['Loss'], label='Loss')
    plt.title(f'Training Progress for {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()
